In [ ]:
from serve import H2OxHandler
from ts.context import Context
import xarray as xr
import json
    
%load_ext autoreload
%autoreload 2

In [ ]:
ctx = Context(
    model_name='kaveri', 
    model_dir='/home/lucas/h2ox-ai/models/', 
    manifest = {'model': 
                {
                    'serializedFile': 'kaveri.pt',
                }
               }, 
    batch_size=8, 
    gpu='0', 
    mms_version=0.1, 
    limit_max_image_pixels=True
)

In [ ]:
inst = H2OxHandler()
inst.initialize(ctx)

In [ ]:
import os, pickle
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import torch

In [ ]:
ds = xr.open_dataset('./../data/data_kaveri.nc')

In [ ]:
preds = xr.open_dataset('./../experiments_2/sacred/147/preds.nc')

In [ ]:
preds

In [ ]:
preds['sim-mean'].sel({'site':'krishnaraja_sagar','date':datetime(2020,10,1)}).cumsum(dim='step')

In [ ]:
trn_sample['krishnaraja_sagar']['y']

In [ ]:
ds

In [ ]:
ds['targets_WATER_VOLUME'].sel({'global_sites':'krishnaraja_sagar','date':pd.date_range('2020-10-01','2020-12-31'),'steps':timedelta(days=0)})

In [ ]:
ds['tigge_t2m'].sel({'date':pd.date_range('2020-07-01','2020-10-01'),'steps':timedelta(days=0),'global_sites':'narayanapura'}).plot()

In [ ]:
sample_data = json.load(open('./../data/kaveri_sample_2020_10_01.json','r'))

In [ ]:
sample_data['2020-10-01']['narayanapura']['x_d']['chirps_precip'][0:5]

In [ ]:
inps = inst.preprocess([sample_data)

In [ ]:
inps['2020-10-01']['narayanapura']['x_d'][:,0:5]

In [ ]:
dl_data = pickle.load(open('./../data/ds_data.pkl','rb'))
dl_meta = pickle.load(open('./../data/ds_meta.pkl','rb'))

In [ ]:
[idx for idx in range(len(dl_meta)) if dl_meta[idx]['date']==np.datetime64(datetime(2020,10,1).isoformat())]

In [ ]:
trn_sample = dl_data[3836]

In [ ]:
trn_sample['narayanapura'].keys()

In [ ]:
sample_data['2020-10-01']['narayanapura']['x_d']['chirps_precip'][1:5]

In [ ]:
trn_sample['narayanapura']['hist_level'][-5:,:]

In [ ]:
inps['2020-10-01']['narayanapura']['hist_level'].cpu().numpy()[:,-5:,:] # precip still berk

In [ ]:
trn_sample['narayanapura']['y'][0:5,:]

In [ ]:
inps['2020-10-01']['narayanapura']['ys'].cpu().numpy()[:,0:5,:] # precip still berk

### Test data

In [ ]:
ds['tigge_t2m'].sel({'date':pd.date_range('2020-07-04','2020-10-01'),'steps':timedelta(days=0),'global_sites':'narayanapura'}).plot() # dataframe

In [ ]:
plt.plot(trn_sample['narayanapura']['x_d'][:,0]) # pkl dataframe

In [ ]:
ds['chirps_precip'].sel({'date':pd.date_range('2020-07-04','2020-10-01'),'steps':timedelta(days=0),'global_sites':'narayanapura'}).plot() # dataframe

In [ ]:
plt.plot(trn_sample['narayanapura']['x_d'][:,1]) # pkl dataframe

**conclusion:** all hist data are back

In [ ]:
### check fut
ds['tigge_tp'].sel({'date':pd.date_range('2020-10-01','2020-10-01'),'steps':[timedelta(days=ii) for ii in range(0,15)],'global_sites':'narayanapura'}).plot() # dataframe

In [ ]:
plt.plot(trn_sample['narayanapura']['x_f'][:,1]) # pkl dataframe

In [ ]:
plt.plot(trn_sample['narayanapura']['x_f'][:,0])

In [ ]:
ds['tigge_t2m'].sel({'date':pd.date_range('2020-10-01','2020-10-01'),'steps':[timedelta(days=ii) for ii in range(1,15)],'global_sites':'narayanapura'}).plot() # dataframe

**conclusion** Forecast seems okay but off by a day

In [ ]:
np.cos(pd.date_range('2020-10-16','2020-12-31').dayofyear / 365. * 2 * np.pi)

In [ ]:
torch.from_numpy(trn_sample['narayanapura']['x_ff'].astype(np.float32))[0:5,:]

**conclusion** Fut also seems okay

In [ ]:
torch.from_numpy(trn_sample['narayanapura']['x_d'].astype(np.float32))[-5:,:] # cos and sin should be switched

In [ ]:
inps['2020-10-01']['narayanapura']['x_d'][0,-5:,:] # temp precip cos sin

In [ ]:
np.sin(pd.date_range('2020-07-01','2020-10-01').dayofyear / 365. * 2 * np.pi)

### test inference

In [ ]:
import torch

In [ ]:
y_hat = inst.inference(inps)

In [ ]:
yy = inst.model(inps['2020-10-01'])

In [ ]:
yy

In [ ]:
y_hat['2020-10-01'][:,:,2].mean(dim=0)#*krs_diff_std

In [ ]:
inst.model = inst.model.eval()

In [ ]:
inst.model(inps['2020-10-01'])

In [ ]:
y_hatb = inst.inference(inps)

In [ ]:
inst.model.load_state_dict(torch.load('./../experiments_2/sacred/147/model_epoch249.pt'))

In [ ]:
trn_sample['krishnaraja_sagar']['y']

In [ ]:
y_hatb['2020-10-01'][:,:,2].mean(dim=0)*krs_diff_std

In [ ]:
preds['sim-mean'].sel({'site':'krishnaraja_sagar','date':datetime(2020,10,1)}) / krs_diff_std

In [ ]:
preds['sim-mean'].sel({'site':'krishnaraja_sagar','date':datetime(2020,10,1)}) / krs_diff_std

In [ ]:
y_hat['2020-10-01'][:,:,2].std(dim=0)

In [ ]:
y_hatb['2020-10-01'][:,:,2].mean(dim=0)# * krs_diff_std

In [ ]:
y_hat['2020-10-01'][0,:,2]#.mean(dim=0)

In [ ]:
results = inst.postprocess(y_hat, inps)

In [ ]:
results['2020-10-01']['mean']

In [ ]:
inst.sites

In [ ]:
inps['2020-10-01']['krishnaraja_sagar']['hist_level']

In [ ]:
var_norms = json.load(open('./../models/kaveri_preprocessing.json','r'))

In [ ]:
var_norms

In [ ]:
krs_diff_std = var_norms['std_norm']['shift_targets_WATER_VOLUME']['std']['krishnaraja_sagar']

In [ ]:
krs_diff_std

In [ ]:
(y_hat['2020-10-01'].amax(dim=0)[:,2]*krs_diff_std).cumsum(dim=0)

In [ ]:
(y_hat['2020-10-01'].amin(dim=0)[:,2]*krs_diff_std).cumsum(dim=0)

In [ ]:
(y_hat['2020-10-01'].mean(dim=0)[:,2]*krs_diff_std).cumsum(dim=0)

In [ ]:
(results['2020-10-01']['lower'][:,2]*krs_diff_std).cumsum(dim=0)

In [ ]:
results['2020-10-01']['mean'][:,2]

In [ ]:
results['2020-10-01']['upper'][:,2]

In [ ]:
y_hat['2020-10-01'].shape

In [ ]:
paths = y_hat['2020-10-01'].cumsum(dim=1)
means = paths.mean(axis=0)
path_stds = paths.std(axis=0)

In [ ]:
y_hat['2020-10-01'][11,:,2]

In [ ]:
paths[0,:,2]

In [ ]:
paths.mean(dim=0)[:,2]

In [ ]:
def reverse_cumsum(arr, dim):
    shp = list(arr.shape)
    shp[dim] = 1
    return torch.diff(arr, dim=dim, prepend=torch.zeros(*list(shp)).to(arr.device))

In [ ]:
ci_upper = means + (2. * path_stds)
ci_lower = means - (2. * path_stds)

means = reverse_cumsum(means, dim=1)
ci_upper = reverse_cumsum(ci_upper, dim=1)
ci_lower = reverse_cumsum(ci_lower, dim=1)

In [ ]:
means[:,2]

In [ ]:
paths = preds.cumsum(dim=2)

    means = paths.mean(axis=0)
    path_stds = paths.std(axis=0)

    # maybe need to go to and back from levels vs. deltas

    ci_upper = means + (ci_std * path_stds)
    ci_lower = means - (ci_std * path_stds)

    means = reverse_cumsum(means, dim=1)
    ci_upper = reverse_cumsum(ci_upper, dim=1)
    ci_lower = reverse_cumsum(ci_lower, dim=1)

In [ ]:
inps = inst.preprocess(sample_data)
    
    #print (inps)
    y_hat = inst.inference(inps)
    
    print ('yhat', y_hat)
    print ('SHAPE')
    print ([(kk,vv.shape) for kk,vv in y_hat.items()])
    
    results = inst.postprocess(y_hat, inps)
    
    print ('RESULTS')
    print (results)